# Анализ данных Algatop

Цель: вычислить формулу расчёта продаж

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 1. Загрузка данных

In [ ]:
# Укажи путь к файлу
DATA_PATH = '../data/algatop_export.csv'  # или .xlsx

# Загрузка
try:
    if DATA_PATH.endswith('.csv'):
        df = pd.read_csv(DATA_PATH, encoding='utf-8')
    else:
        df = pd.read_excel(DATA_PATH)
    print(f'Загружено {len(df)} строк')
except Exception as e:
    print(f'Ошибка: {e}')
    # Попробуй другую кодировку для CSV
    df = pd.read_csv(DATA_PATH, encoding='cp1251')

In [ ]:
# Структура данных
print('Колонки:')
for i, col in enumerate(df.columns):
    print(f'{i+1}. {col} ({df[col].dtype})')

In [ ]:
# Первые строки
df.head(10)

## 2. Маппинг колонок

Заполни названия колонок из твоего файла:

In [ ]:
# ЗАПОЛНИ ЭТИ ПОЛЯ названиями колонок из твоего файла
COLUMNS = {
    'name': 'название',        # Название товара
    'category': 'категория',   # Категория
    'sales': 'продажи',        # Продажи (текущие или за период)
    'reviews': 'отзывы',       # Количество отзывов
    'rating': 'рейтинг',       # Рейтинг товара
    'sellers': 'продавцы',     # Количество продавцов
    'revenue': 'выручка',      # Выручка
    'price': 'цена',           # Цена
}

# Колонки с помесячными продажами (если есть)
MONTHLY_SALES_COLS = [
    # 'янв_2024', 'фев_2024', 'мар_2024', ...
]

## 3. Анализ корреляций

In [ ]:
# Корреляционная матрица
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Корреляция между переменными')
plt.tight_layout()
plt.show()

In [ ]:
# Корреляция с продажами
if COLUMNS['sales'] in df.columns:
    sales_corr = df[numeric_cols].corrwith(df[COLUMNS['sales']]).sort_values(ascending=False)
    print('Корреляция с продажами:')
    print(sales_corr)

## 4. Вычисление коэффициента: продажи / отзывы

In [ ]:
# Главная гипотеза: sales = reviews * coefficient
if COLUMNS['sales'] in df.columns and COLUMNS['reviews'] in df.columns:
    # Фильтруем нули
    valid = df[(df[COLUMNS['reviews']] > 0) & (df[COLUMNS['sales']] > 0)].copy()
    
    valid['coefficient'] = valid[COLUMNS['sales']] / valid[COLUMNS['reviews']]
    
    print(f'Анализ {len(valid)} товаров с отзывами и продажами')
    print(f'\nКоэффициент (sales/reviews):')
    print(f'  Медиана: {valid["coefficient"].median():.2f}')
    print(f'  Среднее: {valid["coefficient"].mean():.2f}')
    print(f'  Мин: {valid["coefficient"].min():.2f}')
    print(f'  Макс: {valid["coefficient"].max():.2f}')
    
    # Распределение
    plt.figure(figsize=(10, 4))
    plt.hist(valid['coefficient'].clip(0, 100), bins=50, edgecolor='black')
    plt.xlabel('Коэффициент (sales/reviews)')
    plt.ylabel('Количество товаров')
    plt.title('Распределение коэффициента')
    plt.axvline(valid['coefficient'].median(), color='red', linestyle='--', label=f'Медиана: {valid["coefficient"].median():.1f}')
    plt.legend()
    plt.show()

## 5. Коэффициенты по категориям

In [ ]:
# Разбивка по категориям
if COLUMNS['category'] in df.columns and 'coefficient' in valid.columns:
    cat_stats = valid.groupby(COLUMNS['category']).agg({
        'coefficient': ['median', 'mean', 'count'],
        COLUMNS['sales']: 'sum',
        COLUMNS['reviews']: 'sum'
    }).round(2)
    
    cat_stats.columns = ['coef_median', 'coef_mean', 'count', 'total_sales', 'total_reviews']
    cat_stats = cat_stats.sort_values('total_sales', ascending=False)
    
    print('Коэффициенты по категориям:')
    display(cat_stats.head(20))

## 6. Линейная регрессия

In [ ]:
# Попытка построить формулу через ML
feature_cols = [COLUMNS['reviews'], COLUMNS['rating'], COLUMNS['sellers']]
feature_cols = [c for c in feature_cols if c in df.columns]

if feature_cols and COLUMNS['sales'] in df.columns:
    X = df[feature_cols].fillna(0)
    y = df[COLUMNS['sales']].fillna(0)
    
    # Фильтр нулей
    mask = (X.sum(axis=1) > 0) & (y > 0)
    X, y = X[mask], y[mask]
    
    # Линейная регрессия
    lr = LinearRegression()
    lr.fit(X, y)
    
    y_pred = lr.predict(X)
    r2 = r2_score(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    
    print('='*50)
    print('ЛИНЕЙНАЯ РЕГРЕССИЯ')
    print('='*50)
    print(f'R² score: {r2:.4f}')
    print(f'MAE: {mae:.2f}')
    print(f'\nФормула:')
    print(f'  sales = {lr.intercept_:.2f}')
    for col, coef in zip(feature_cols, lr.coef_):
        sign = '+' if coef > 0 else '-'
        print(f'         {sign} {abs(coef):.4f} * {col}')

## 7. Random Forest (нелинейные зависимости)

In [ ]:
# Random Forest для поиска важности признаков
if feature_cols and len(X) > 100:
    rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X, y)
    
    y_pred_rf = rf.predict(X)
    r2_rf = r2_score(y, y_pred_rf)
    
    print('='*50)
    print('RANDOM FOREST')
    print('='*50)
    print(f'R² score: {r2_rf:.4f}')
    print(f'\nВажность признаков:')
    for col, imp in sorted(zip(feature_cols, rf.feature_importances_), key=lambda x: -x[1]):
        print(f'  {col}: {imp:.4f}')

## 8. Итоговая формула

In [ ]:
# Сохранение коэффициентов
FORMULA = {
    'global_coefficient': valid['coefficient'].median() if 'coefficient' in valid.columns else None,
    'category_coefficients': cat_stats['coef_median'].to_dict() if 'cat_stats' in dir() else {},
    'linear_regression': {
        'intercept': lr.intercept_ if 'lr' in dir() else None,
        'coefficients': dict(zip(feature_cols, lr.coef_)) if 'lr' in dir() else {}
    }
}

print('='*50)
print('ИТОГОВАЯ ФОРМУЛА')
print('='*50)
print(f'\nПростая формула:')
print(f'  sales = reviews * {FORMULA["global_coefficient"]:.1f}')
print(f'\nИли по категориям (для точности):')
for cat, coef in list(FORMULA['category_coefficients'].items())[:10]:
    print(f'  {cat}: sales = reviews * {coef:.1f}')

In [ ]:
# Экспорт коэффициентов в JSON
import json

with open('../models/coefficients.json', 'w', encoding='utf-8') as f:
    json.dump(FORMULA, f, ensure_ascii=False, indent=2, default=str)

print('Коэффициенты сохранены в ../models/coefficients.json')